## Game: Minimax Search

we are going to decide best move in Othello game by runnig minimax algorithm, then we are going to apply alpha-beta pruning and see its effect on time and seen states

In [16]:
import random
import time
import turtle
from copy import deepcopy
from timeit import default_timer as timer

In [17]:
HUMAN  = 1
CPU  = -1

DEPTH_LEAF = 2
ENDGAME_LEAF = 3

INF = 1000

In [18]:
class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()


All changes:
- get_valid_moves and make_move methods now works with given board if no board given, default game board(currently playing board) will be used.
- A nested Minimax class added, encapsulating every state's informations in a class and using it to access children and parents can be helpful

Q1 : calc_eval calculate evaluation of a leaf, we have 2 kind of leaves, depth-limited leaves and end-game leaves, evaluation of an end-game leaf must be so great or so low, because the game is finished and we got the winner thus it has more proirity comparing to depth-limited leaves, when we reach limit of depth we produce a depth-limited leaf which calculate evaluation by subtracting numbers of discs, a good act can be giving weight to board corners and edges, corner discs can not be changed and edge discs are game-changing in Othello, as we analyse minimax algoritm, AI decides edge and corner discs however it results in less disc changes, but in deeper depthes it has a really good affect to change huge amount of discs' color. 

Notes:
- if there is no more valid moves, that state is end-game leaf, and has a great effect on evaluation.
- end-game leaf does not affects normally in game tie
- every state updates its parent and children, saving them can be helpful in further analyses.
- alpha and beta is calculated for every state, I using simple a >= b condition instead of recursive way.
- prune is conditionally, both way will be calculated.

In [19]:
class Othello:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player, board = None):
        if board == None:
            board = self.board

        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)


    def make_move(self, player, move, board = None):
        if board == None:
            board = self.board

        i, j = move
        board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    class Minimax:
        def __init__(self, board, player, parent, prev_move, next_move, alpha, beta):
            self.board = board
            self.player = player
            self.parent = parent
            self.prev_move = prev_move
            self.next_move = next_move
            self.alpha = alpha
            self.beta = beta
            self.children = []

            if(player == HUMAN):
                self.evaluation = -INF
            elif(player == CPU):
                self.evaluation = INF

        def __lt__(self, other):
            return self.evaluation < other.evaluation

        def calc_eval(self, leaf_type):
            diff = sum([row.count(1) for row in self.board]) - sum([row.count(-1) for row in self.board])
            if(leaf_type == DEPTH_LEAF or diff == 0):
                self.evaluation = diff
                self.alpha = diff
                self.beta = diff
            elif(diff < 0):
                self.evaluation = -36
                self.alpha = -36
                self.beta = -36
            elif(diff > 0):
                self.evaluation = 36
                self.alpha = 36
                self.beta = 36
            
        
        def pruning(self):
            return(self.alpha >= self.beta)

        def handle_parent(self, leaf_type = 0):
            curr = self
            par = self.parent

            if(leaf_type != 0):
                self.calc_eval(leaf_type)

            if par == None:
                return

            if(par.player == HUMAN and curr.evaluation > par.evaluation):
                par.evaluation = curr.evaluation
                par.next_move = curr.prev_move
                par.alpha = max(par.alpha, par.evaluation)
            elif(par.player == CPU and curr.evaluation < par.evaluation):
                par.evaluation = curr.evaluation
                par.next_move = curr.prev_move
                par.beta = min(par.beta, par.evaluation)        
        

    def minimax_search(self, curr_state, depth):
        
        if(depth < self.minimax_depth):
            moves = self.get_valid_moves(curr_state.player, curr_state.board)
            if not moves:
                curr_state.player = -curr_state.player
                curr_state.evaluation = -curr_state.evaluation
                moves = self.get_valid_moves(curr_state.player, curr_state.board)
                if not moves:
                    curr_state.handle_parent(ENDGAME_LEAF)
                    return

            for move in moves:
                
                if(self.prune):
                    if(curr_state.pruning()):
                        return

                new_board = deepcopy(curr_state.board)
                self.make_move(curr_state.player, move, new_board)
                new_state = self.Minimax(new_board, -curr_state.player, curr_state, move, None, curr_state.alpha, curr_state.beta)
                curr_state.children.append(new_state)

                self.minimax_search(new_state, depth + 1)
            curr_state.handle_parent()
            
        else:
            curr_state.handle_parent(DEPTH_LEAF)

    def get_human_move(self):
        curr = self.Minimax(deepcopy(self.board), self.current_turn, None, None, None, -INF, INF)
        self.minimax_search(curr, 0)
        return curr.next_move
        
    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    def play(self):
        winner = None
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(2)
        winner = self.get_winner()
        return winner

In [20]:
print('without pruning results:\n')
times = 120
depthes = [1,3,5]
for depth in depthes:
    wins = 0
    print('depth = ',depth)
    start = timer()
    for i in range(times):
        game = Othello(ui=False, minimax_depth=depth, prune=False)
        if(game.play() == 1):
            wins += 1
        del game
    end = timer()
    print('win_rate =', (wins/times) * 100)
    print('average game time = ',(end - start)/ times,'\n')

without pruning results:

depth =  1
win_rate = 76.66666666666667
average game time =  0.006697195833333088 

depth =  3
win_rate = 82.5
average game time =  0.14107655249999976 

depth =  5
win_rate = 99.16666666666667
average game time =  5.703711637500001 



In [21]:
print('with pruning results:\n')
times = 120
depthes = [1,3,5,7]
for depth in depthes:
    wins = 0
    print('depth = ',depth)
    start = timer()
    for i in range(times):
        game = Othello(ui=False, minimax_depth=depth, prune=True)
        if(game.play() == 1):
            wins += 1
        del game
    end = timer()
    print('win_rate =', (wins/times) * 100)
    print('average game time = ',(end - start)/ times,'\n')

with pruning results:

depth =  1
win_rate = 73.33333333333333
average game time =  0.006702328333333677 

depth =  3
win_rate = 86.66666666666667
average game time =  0.06207985583333387 

depth =  5
win_rate = 91.66666666666666
average game time =  0.6425725116666664 

depth =  7
win_rate = 99.16666666666667
average game time =  7.245271075 



Q2: as the depth increases, winning rate and average game time increases. in higher depthes we must analyse more states and game tree will be more complicated, but we predict more moves and choose a better decide which causes more wins.

Q3: some kind of acts can be done to have more pruning, like iterate more high-amount max nodes and less-amount min nodes first, but that causes an overload to program which may reduce efficiency.

Q4: every allowed moves and its effect to board and discs is a branching factor, first we have limited moves while in next moves we have more decisions, but from second half of game decisions gets less, by analysing children of nodes I guess it has a normal distribution.

Q5: pruning cuts no more useful children which analysing them does not change answer of minimax search, imagine we have a min father in our path to a node if we assign a higher amount of that node to a max node, the other children of that nodes are no more longer helpful, because we already found a bigger amount sibbling.(vice versa)

Q6: if the opponent does not take best action, we can use expectimax algorithm insted. using minimax against a random-based(or not completely efficient) opponent can prevent us to reach a higher amount leaf, according the fact we found best way in worst-case, but in avg-case we can reach a better score.

in this case which opponent is completely random we can use uniform distribution and give sum of 1/allowed_moves * child.evaluation to a node as its evaluation isntead of minimum evaluation.